**Data Acquisition and Preprocessing**

**BITCOIN**

In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd

# Fetch full historical data for Bitcoin (BTC-USD)
btc = yf.download("BTC-USD", start="2013-01-01", end="2021-12-31", interval="1d")

# Clean up and reset
btc = btc.reset_index()[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
btc.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Open,High,Low,Close,Volume
Ticker,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


In [ ]:
print(btc.columns)


MultiIndex([(  'Date',        ''),
            (  'Open', 'BTC-USD'),
            (  'High', 'BTC-USD'),
            (   'Low', 'BTC-USD'),
            ( 'Close', 'BTC-USD'),
            ('Volume', 'BTC-USD')],
           names=['Price', 'Ticker'])


In [ ]:
# Flatten MultiIndex columns by joining levels with underscore (or space)
btc.columns = ['_'.join(filter(None, col)) for col in btc.columns]

# Check new column names
print(btc.columns)

# Save to Excel
btc.to_excel("bitcoin_data.xlsx", index=False)

# For Google Colab: trigger download
from google.colab import files
files.download("bitcoin_data.xlsx")


Index(['Date', 'Open_BTC-USD', 'High_BTC-USD', 'Low_BTC-USD', 'Close_BTC-USD',
       'Volume_BTC-USD'],
      dtype='object')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install yfinance
!pip install yfinance --quiet

# Import libraries
import yfinance as yf
import pandas as pd
from google.colab import files

# Download Ethereum historical data from Yahoo Finance
eth = yf.download("ETH-USD", start="2013-01-01", end="2021-12-31", interval="1d", group_by='ticker')

# Reset index
eth = eth.reset_index()

# Flatten MultiIndex columns
if isinstance(eth.columns, pd.MultiIndex):
    eth.columns = ['_'.join(filter(None, col)) for col in eth.columns]
else:
    eth.columns = [str(col) for col in eth.columns]

eth.head()

[*********************100%***********************]  1 of 1 completed


,Date,ETH-USD_Open,ETH-USD_High,ETH-USD_Low,ETH-USD_Close,ETH-USD_Volume
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,893249984
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,885985984
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,842300992
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,1613479936
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,1041889984


In [ ]:
# Save to Excel
eth.to_excel("ethereum_data.xlsx", index=False)

# Trigger download in Google Colab
files.download("ethereum_data.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load Bitcoin data into DataFrame
btc = pd.read_excel("/content/bitcoin_data.xlsx")

btc.head()

,Date,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Close_BTC-USD,Volume_BTC-USD
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


In [ ]:
# Function to clean dataset: parse dates, handle missing values
def clean_data(df):
    # Convert 'Date' column to datetime type, coercing errors to NaT
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    # Drop rows where 'Date' could not be parsed
    df = df.dropna(subset=['Date'])
    # Sort dataframe by date ascending
    df = df.sort_values('Date').reset_index(drop=True)

    # Fill missing prices by forward-fill then backward-fill to avoid gaps
    price_cols = ['Open_BTC-USD', 'High_BTC-USD', 'Low_BTC-USD', 'Close_BTC-USD']
    for col in price_cols:
        if col in df.columns:
            df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

    # Fill missing volumes with zero (assumed no volume)
    if 'Volume' in df.columns:
        df['Volume_BTC-USD'] = df['Volume_BTC-USD'].fillna(0)

    return df

In [ ]:
# Function to remove anomalies: filter out invalid price or volume entries
def remove_anomalies(df):
    conditions = (
        (df['Open_BTC-USD'] > 0) &
        (df['High_BTC-USD'] > 0) &
        (df['Low_BTC-USD'] > 0) &
        (df['Close_BTC-USD'] > 0) &
        (df['Volume_BTC-USD'] >= 0)
    )
    return df[conditions].reset_index(drop=True)


In [ ]:
def feature_engineering(df):
    """
    Adds new financial features to the DataFrame without applying normalization.
    """
    df['Volatility'] = (df['High_BTC-USD'] - df['Low_BTC-USD']) / df['Close_BTC-USD']
    df['MA_7'] = df['Close_BTC-USD'].rolling(window=7).mean()
    df['MA_30'] = df['Close_BTC-USD'].rolling(window=30).mean()
    df['Daily_Return'] = df['Close_BTC-USD'].pct_change()

    df.fillna(method='bfill', inplace=True)

    # Add a placeholder sentiment score if not already present
    if 'Sentiment_Score' not in df.columns:
        df['Sentiment_Score'] = 0

    return df

In [ ]:
# Clean and remove anomalies
btc = clean_data(btc)
btc = remove_anomalies(btc)

<ipython-input-23-7455a55e07b5>:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')


In [ ]:
# Apply feature engineering (unnormalized)
btc = feature_engineering(btc)

# Save to Excel (unnormalized version)
btc.to_excel("bitcoin_unnormalized.xlsx", index=False)

# Download in Colab
from google.colab import files
files.download("bitcoin_unnormalized.xlsx")


<ipython-input-29-5de841c48b79>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalize_numeric_columns(df, exclude_columns=['Date']):
    """
    Normalizes all numeric columns in the DataFrame, excluding specified columns.
    """
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cols_to_normalize = [col for col in numeric_cols if col not in exclude_columns]

    scaler = MinMaxScaler()
    df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
    return df


In [ ]:
# Normalize the BTC dataset
btc_normalized = btc.copy()
btc_normalized = normalize_numeric_columns(btc_normalized, exclude_columns=['Date'])

# Save to Excel (normalized version)
btc_normalized.to_excel("bitcoin_normalized.xlsx", index=False)

# Download in Colab
files.download("bitcoin_normalized.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**ETHEREUM**

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load Ethereum data
eth = pd.read_excel("/content/ethereum_data.xlsx")

eth.head()

,Date,ETH-USD_Open,ETH-USD_High,ETH-USD_Low,ETH-USD_Close,ETH-USD_Volume
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,893249984
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,885985984
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,842300992
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,1613479936
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,1041889984


In [ ]:
# --- CLEANING FUNCTION ---
def clean_data_eth(df):
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    price_cols = ['ETH-USD_Open', 'ETH-USD_High', 'ETH-USD_Low', 'ETH-USD_Close']
    for col in price_cols:
        if col in df.columns:
            df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

    if 'ETH-USD_Volume' in df.columns:
        df['ETH-USD_Volume'] = df['ETH-USD_Volume'].fillna(0)

    return df

In [ ]:
# --- ANOMALY REMOVAL ---
def remove_anomalies_eth(df):
    conditions = (
        (df['ETH-USD_Open'] > 0) &
        (df['ETH-USD_High'] > 0) &
        (df['ETH-USD_Low'] > 0) &
        (df['ETH-USD_Close'] > 0) &
        (df['ETH-USD_Volume'] >= 0)
    )
    return df[conditions].reset_index(drop=True)

In [ ]:
# --- FEATURE ENGINEERING ---
def feature_engineering_eth(df):
    df['Volatility'] = (df['ETH-USD_High'] - df['ETH-USD_Low']) / df['ETH-USD_Close']
    df['MA_7'] = df['ETH-USD_Close'].rolling(window=7).mean()
    df['MA_30'] = df['ETH-USD_Close'].rolling(window=30).mean()
    df['Daily_Return'] = df['ETH-USD_Close'].pct_change()
    df.fillna(method='bfill', inplace=True)

    if 'Sentiment_Score' not in df.columns:
        df['Sentiment_Score'] = 0

    return df

In [ ]:
# --- NORMALIZATION FUNCTION ---
def normalize_numeric_columns(df, exclude_columns=['Date']):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cols_to_normalize = [col for col in numeric_cols if col not in exclude_columns]

    scaler = MinMaxScaler()
    df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
    return df


In [ ]:
# Clean and remove anomalies
eth = clean_data_eth(eth)
eth = remove_anomalies_eth(eth)

<ipython-input-35-e06dbf7e3b70>:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')


In [ ]:
# Feature engineering (unnormalized)
eth = feature_engineering_eth(eth)

<ipython-input-37-6ed0d07f2f71>:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [ ]:
# Save unnormalized
eth.to_excel("ethereum_unnormalized.xlsx", index=False)
files.download("ethereum_unnormalized.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Normalize and save normalized
eth_normalized = eth.copy()
eth_normalized = normalize_numeric_columns(eth_normalized, exclude_columns=['Date'])

In [ ]:
eth_normalized.to_excel("ethereum_normalized.xlsx", index=False)
files.download("ethereum_normalized.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>